In [13]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [14]:
def sentiment_to_score(sentiment):
    #감성 → 점수 변환 (긍정=+1, 부정=-1, 중립=0)
    sentiment = str(sentiment).strip()
    if '긍정' in sentiment:
        return 1.0
    elif '부정' in sentiment:
        return -1.0
    else:
        return 0.0

def calculate_sentiment_index(positive, negative, total):
    #감성 지수 계산: (긍정 - 부정) / 전체
    if total == 0:
        return 0.0
    return (positive - negative) / total

In [15]:
# 데이터 로드 - skiprows 제거하고 직접 확인
df = pd.read_csv('../data/sentiment/keyword_sentiment.csv')

print(f"데이터 로드: {len(df)}개 뉴스")
print(f"\n상위 5개 데이터:")
print(df.head())

print(f"\n컬럼 목록:")
for i, col in enumerate(df.columns):
    print(f"{i}: '{col}'")

데이터 로드: 20개 뉴스

상위 5개 데이터:
   number  crawled_news_id                                     title  \
0       1            16419       KT·롯데카드 청문회-13 "해킹은 사고지만 은폐·축소는 범죄"   
1       2            16459      로밍부터 OTT까지 한 번에 SKT, 추석 연휴 특별 혜택 대공개   
2       3            16534  추석 연휴를 앞두고 고성능 전기차 시장 선점 나선 아우디·현대차 나...   
3       4            16578       KB금융, '아동 야간 돌봄' 강화 적극 지원...복지부와 맞손   
4       5            16581                  KB금융, 야간 연장돌봄사업에 60억원 지원   

           pub_date                                               text  \
0  2025/10/05 11:39  김영섭 KT 대표가 더불어민주당 노종면 의원의 질의를 듣고 있다. 24 유튜브 갈무...   
1  2025/10/05 10:02  SK텔레콤이 추석 연휴를 맞아 로밍, 멤버십, 구독 서비스 T 우주 , 체험형 공간...   
2   2025/10/05 7:00  현대차 2026 쏘나타 디 엣지 현대차 아이오닉 5 N '에센셜' 트림 이번 주는 ...   
3   2025/10/05 1:40  KB금융그룹이 보건복지부와 손잡고 '아동 야간 돌봄 공백 해소'를 위해 뛴다. 5일...   
4   2025/10/05 0:24  야감 생업 부모 걱정 해소 자정까지 운영 연장돌봄 지원 양종희 KB금융그룹 회장과 ...   

  related_stocks                                       keywords sentiment  \
0             KT  

In [ ]:
# 컬럼명 설정 (문자열)
stock_col_name = 'related_stocks'
sentiment_col_name = 'stocks_sentiment_str'
keywords_col_name = 'keywords'
keyword_sentiment_col_name = 'keywords_sentiment_str'

# 종목별 집계
stock_data = defaultdict(lambda: {'positive': 0, 'negative': 0, 'neutral': 0, 'total': 0})
keyword_data = defaultdict(lambda: {'positive': 0, 'negative': 0, 'neutral': 0, 'total': 0})

for _, row in df.iterrows():
    # iloc 대신 컬럼명 직접 사용
    stock_str = str(row[stock_col_name]).strip()
    sentiment = str(row[sentiment_col_name]).strip()
    keywords_str = str(row[keywords_col_name]).strip()
    keyword_sentiment = str(row[keyword_sentiment_col_name]).strip()
    
    # 유효한 종목만 처리
    if stock_str and stock_str != 'nan' and stock_str != '':
        # 컴마로 분리 (여러 종목이 있는 경우)
        stocks = [s.strip() for s in stock_str.split(',')]
        
        # 각 종목별로 집계
        for stock in stocks:
            if stock:  # 빈 문자열 제외
                stock_data[stock]['total'] += 1
                
                if '긍정' in sentiment:
                    stock_data[stock]['positive'] += 1
                elif '부정' in sentiment:
                    stock_data[stock]['negative'] += 1
                else:
                    stock_data[stock]['neutral'] += 1

        keywords = [s.strip() for s in keywords_str.split(',')]
        keyword_sentiments = [s.strip() for s in keyword_sentiment.split(',')]

        for keyword, keyword_sentiment in zip(keywords, keyword_sentiments):
            if keyword:
                keyword_data[keyword]['total'] += 1

                if '긍정' in keyword_sentiment:
                    keyword_data[keyword]['positive'] += 1
                elif '부정' in keyword_sentiment:
                    keyword_data[keyword]['negative'] += 1
                else:
                    keyword_data[keyword]['neutral'] += 1

# 감성 지수 계산
results = []
for stock, data in stock_data.items():
    sentiment_index = calculate_sentiment_index(
        data['positive'], data['negative'], data['total']
    )
    results.append({
        'stock': stock,
        'total': data['total'],
        'positive': data['positive'],
        'negative': data['negative'],
        'neutral': data['neutral'],
        'sentiment_index': sentiment_index
    })

results = sorted(results, key=lambda x: x['sentiment_index'], reverse=True)
print(f"계산 완료: {len(results)}개 종목")

keywords_results = []
for keyword, data in keyword_data.items():
    sentiment_index = calculate_sentiment_index(
        data['positive'], data['negative'], data['total']
    )
    keywords_results.append({
        'keyword': keyword,
        'total': data['total'],
        'positive': data['positive'],
        'negative': data['negative'],
        'neutral': data['neutral'],
        'sentiment_index': sentiment_index
    })

keywords_results = sorted(keywords_results, key=lambda x: x['sentiment_index'], reverse=True)
print(f"계산 완료: {len(keywords_results)}개 키워드")


계산 완료: 15개 종목
계산 완료: 102개 키워드


In [21]:
print("\n" + "="*70)
print("종목별 감성 지수")
print("="*70)

for rank, r in enumerate(results, 1):
    print(f"{rank}. {r['stock']:<20} | "
          f"감성지수: {r['sentiment_index']:+.3f} | "
          f"긍정: {r['positive']}  부정: {r['negative']}  전체: {r['total']}")


종목별 감성 지수
1. SK텔레콤                | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
2. KB금융                 | 감성지수: +1.000 | 긍정: 7  부정: 0  전체: 7
3. CJ ENM               | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
4. 전진건설로봇               | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
5. 미래에셋증권               | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
6. NAVER                | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
7. 안랩                   | 감성지수: +1.000 | 긍정: 2  부정: 0  전체: 2
8. LG전자                 | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
9. CJ                   | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
10. 롯데쇼핑                 | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
11. 현대차                  | 감성지수: +0.000 | 긍정: 1  부정: 1  전체: 2
12. 카카오                  | 감성지수: +0.000 | 긍정: 0  부정: 0  전체: 1
13. KT                   | 감성지수: -1.000 | 긍정: 0  부정: 2  전체: 2
14. LG생활건강               | 감성지수: -1.000 | 긍정: 0  부정: 1  전체: 1
15. 아모레퍼시픽               | 감성지수: -1.000 | 긍정: 0  부정: 1  전체: 1


In [54]:
results_df = pd.DataFrame(results)
print("\n데이터프레임으로 변환 완료")
results_df.head()


데이터프레임으로 변환 완료


,stock,total,positive,negative,neutral,sentiment_index
0,SK텔레콤,1,1,0,0,1.0
1,KB금융,7,7,0,0,1.0
2,CJ ENM,1,1,0,0,1.0
3,전진건설로봇,1,1,0,0,1.0
4,NAVER,1,1,0,0,1.0


In [58]:
results_df.to_csv('../data/sentiment/sentiment_index.csv', index=False, encoding='utf-8-sig')
print("저장 완료: sentiment_index.csv")

저장 완료: sentiment_index.csv


In [23]:
print("\n" + "="*70)
print("키워드별 감성 지수")
print("="*70)

for rank, r in enumerate(keywords_results, 1):
    print(f"{rank}. {r['keyword']:<20} | "
          f"감성지수: {r['sentiment_index']:+.3f} | "
          f"긍정: {r['positive']}  부정: {r['negative']}  전체: {r['total']}")


키워드별 감성 지수
1. 로밍                   | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
2. 멤버십                  | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
3. T 우주                 | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
4. T팩토리 성수              | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
5. OTT                  | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
6. 할인                   | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
7. SK텔레콤                | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
8. 쏘나타                  | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
9. 트림                   | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
10. 출시                   | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
11. 사양                   | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
12. 아동 야간 돌봄             | 감성지수: +1.000 | 긍정: 5  부정: 0  전체: 5
13. 사회적 안전망              | 감성지수: +1.000 | 긍정: 2  부정: 0  전체: 2
14. 민관 협력                | 감성지수: +1.000 | 긍정: 1  부정: 0  전체: 1
15. KB금융그룹               | 감성지수: +1.000 | 긍정: 6  부정: 0  전체: 6
16. KB국민은행               | 감성지수: +1.000 | 긍정: 3  부정: 0  전체: 3
17. 안

In [24]:
results_df = pd.DataFrame(keywords_results)
print("\n데이터프레임으로 변환 완료")
results_df.head()


데이터프레임으로 변환 완료


,keyword,total,positive,negative,neutral,sentiment_index
0,로밍,1,1,0,0,1.0
1,멤버십,1,1,0,0,1.0
2,T 우주,1,1,0,0,1.0
3,T팩토리 성수,1,1,0,0,1.0
4,OTT,1,1,0,0,1.0


In [25]:
results_df.to_csv('../data/sentiment/keywords_sentiment_index.csv', index=False, encoding='utf-8-sig')
print("저장 완료: keywords_sentiment_index.csv")

저장 완료: keywords_sentiment_index.csv


## 시간 가중치 적용 (미완성)

In [65]:
from datetime import datetime, timedelta

def calculate_time_weight(pub_date, reference_date=None, decay_rate=0.1):
    """
    시간 가중치 계산
    
    Args:
        pub_date: 뉴스 발행일 (datetime 또는 문자열)
        reference_date: 기준일 (None이면 오늘)
        decay_rate: 감쇠율 (0.1 = 하루당 10% 감소)
    
    Returns:
        float: 시간 가중치 (0.0 ~ 1.0)
    """
    if reference_date is None:
        reference_date = datetime.now()
    
    # 문자열이면 datetime으로 변환
    if isinstance(pub_date, str):
        try:
            # 여러 날짜 형식 시도
            for fmt in ['%Y/%m/%d %H:%M', '%Y-%m-%d %H:%M', '%Y/%m/%d', '%Y-%m-%d']:
                try:
                    pub_date = datetime.strptime(pub_date, fmt)
                    break
                except ValueError:
                    continue
        except:
            return 1.0  # 파싱 실패시 기본 가중치
    
    # 날짜 차이 계산
    days_diff = (reference_date - pub_date).days
    
    # 음수면 미래 날짜 (기본 가중치)
    if days_diff < 0:
        return 1.0
    
    # 시간 가중치 계산: weight = max(0.3, 1.0 - days_diff * decay_rate)
    # 최소 0.3까지만 감소 (너무 오래된 뉴스도 최소한의 가치)
    weight = max(0.3, 1.0 - days_diff * decay_rate)
    
    return weight


def calculate_weighted_sentiment_index(df, stock_col, sentiment_col, date_col, reference_date=None):
    """
    시간 가중치를 적용한 종목별 감성 지수 계산
    
    Returns:
        list: 종목별 결과 (일반 감성지수 + 가중 감성지수)
    """
    from collections import defaultdict
    
    stock_data = defaultdict(lambda: {
        'positive': 0,
        'negative': 0,
        'neutral': 0,
        'total': 0,
        'weighted_scores': [],
        'weights': []
    })
    
    for _, row in df.iterrows():
        stock_str = str(row[stock_col]).strip()
        sentiment = str(row[sentiment_col]).strip()
        pub_date = row[date_col]
        
        if stock_str and stock_str != 'nan' and stock_str != '':
            # 컴마로 분리
            stocks = [s.strip() for s in stock_str.split(',')]
            
            # 시간 가중치 계산
            time_weight = calculate_time_weight(pub_date, reference_date)
            
            # 감성 점수
            if '긍정' in sentiment:
                sentiment_score = 1.0
            elif '부정' in sentiment:
                sentiment_score = -1.0
            else:
                sentiment_score = 0.0
            
            # 각 종목별로 집계
            for stock in stocks:
                if stock:
                    stock_data[stock]['total'] += 1
                    
                    if sentiment_score > 0:
                        stock_data[stock]['positive'] += 1
                    elif sentiment_score < 0:
                        stock_data[stock]['negative'] += 1
                    else:
                        stock_data[stock]['neutral'] += 1
                    
                    # 가중치 적용
                    stock_data[stock]['weighted_scores'].append(sentiment_score * time_weight)
                    stock_data[stock]['weights'].append(time_weight)
    
    # 결과 계산
    results = []
    for stock, data in stock_data.items():
        # 일반 감성 지수
        sentiment_index = calculate_sentiment_index(
            data['positive'], data['negative'], data['total']
        )
        
        # 가중 감성 지수
        if sum(data['weights']) > 0:
            weighted_sentiment_index = sum(data['weighted_scores']) / sum(data['weights'])
        else:
            weighted_sentiment_index = 0.0
        
        results.append({
            'stock': stock,
            'total': data['total'],
            'positive': data['positive'],
            'negative': data['negative'],
            'neutral': data['neutral'],
            'sentiment_index': sentiment_index,
            'weighted_sentiment_index': weighted_sentiment_index,
            'avg_weight': sum(data['weights']) / len(data['weights']) if data['weights'] else 0
        })
    
    return sorted(results, key=lambda x: x['weighted_sentiment_index'], reverse=True)

In [66]:
# 시간 가중치 적용 감성 지수 계산
date_col = 'pub_date'  # 날짜 컬럼명 (실제 컬럼명으로 변경)

weighted_results = calculate_weighted_sentiment_index(
    df, 
    stock_col_name, 
    sentiment_col_name, 
    date_col
)

print("="*90)
print("📊 시간 가중치 적용 종목별 감성 지수")
print("="*90)
print(f"\n{'순위':<5} {'종목':<20} {'뉴스':<6} {'일반지수':<10} {'가중지수':<10} {'평균가중치':<10}")
print("-"*90)

for rank, r in enumerate(weighted_results, 1):
    emoji = "🟢" if r['weighted_sentiment_index'] > 0.3 else (
        "🔴" if r['weighted_sentiment_index'] < -0.3 else "🟡"
    )
    
    print(f"{rank:<5} {emoji} {r['stock']:<18} {r['total']:<6} "
          f"{r['sentiment_index']:+.3f}     {r['weighted_sentiment_index']:+.3f}     "
          f"{r['avg_weight']:.3f}")

📊 시간 가중치 적용 종목별 감성 지수

순위    종목                   뉴스     일반지수       가중지수       평균가중치     
------------------------------------------------------------------------------------------
1     🟢 SK텔레콤              1      +1.000     +1.000     0.300
2     🟢 KB금융               7      +1.000     +1.000     0.300
3     🟢 CJ ENM             1      +1.000     +1.000     0.300
4     🟢 전진건설로봇             1      +1.000     +1.000     0.300
5     🟢 NAVER              1      +1.000     +1.000     0.300
6     🟢 카카오                1      +1.000     +1.000     0.300
7     🟢 미래에셋증권             1      +1.000     +1.000     0.300
8     🟢 안랩                 2      +1.000     +1.000     0.300
9     🟢 LG전자               1      +1.000     +1.000     0.300
10    🟢 CJ                 1      +1.000     +1.000     0.300
11    🟢 롯데쇼핑               1      +1.000     +1.000     0.300
12    🔴 현대차                2      -0.500     -0.500     0.300
13    🔴 KT                 2      -1.000     -1.000     0.300
14    🔴 LG생활건

In [67]:
# 일반 vs 가중 감성 지수 비교
print("\n" + "="*90)
print("📈 일반 감성 지수 vs 가중 감성 지수 비교 (TOP 10)")
print("="*90)

comparison_df = pd.DataFrame(weighted_results[:10])
comparison_df['차이'] = comparison_df['weighted_sentiment_index'] - comparison_df['sentiment_index']

print(comparison_df[['stock', 'sentiment_index', 'weighted_sentiment_index', '차이', 'avg_weight']])


📈 일반 감성 지수 vs 가중 감성 지수 비교 (TOP 10)
    stock  sentiment_index  weighted_sentiment_index   차이  avg_weight
0   SK텔레콤              1.0                       1.0  0.0         0.3
1    KB금융              1.0                       1.0  0.0         0.3
2  CJ ENM              1.0                       1.0  0.0         0.3
3  전진건설로봇              1.0                       1.0  0.0         0.3
4   NAVER              1.0                       1.0  0.0         0.3
5     카카오              1.0                       1.0  0.0         0.3
6  미래에셋증권              1.0                       1.0  0.0         0.3
7      안랩              1.0                       1.0  0.0         0.3
8    LG전자              1.0                       1.0  0.0         0.3
9      CJ              1.0                       1.0  0.0         0.3


In [ ]:
# 등급 함수 정의
def get_grade(index):
    """감성 지수 → 등급 변환"""
    if index > 0.5:
        return "매우 긍정적"
    elif index > 0.2:
        return "긍정적"
    elif index > -0.2:
        return "중립적"
    elif index > -0.5:
        return "부정적"
    else:
        return "매우 부정적"

✅ 저장 완료: weighted_sentiment_index.csv


,stock,total,positive,negative,neutral,sentiment_index,weighted_sentiment_index,avg_weight,등급
0,SK텔레콤,1,1,0,0,1.0,1.0,0.3,매우 긍정적
1,KB금융,7,7,0,0,1.0,1.0,0.3,매우 긍정적
2,CJ ENM,1,1,0,0,1.0,1.0,0.3,매우 긍정적
3,전진건설로봇,1,1,0,0,1.0,1.0,0.3,매우 긍정적
4,NAVER,1,1,0,0,1.0,1.0,0.3,매우 긍정적
5,카카오,1,1,0,0,1.0,1.0,0.3,매우 긍정적
6,미래에셋증권,1,1,0,0,1.0,1.0,0.3,매우 긍정적
7,안랩,2,2,0,0,1.0,1.0,0.3,매우 긍정적
8,LG전자,1,1,0,0,1.0,1.0,0.3,매우 긍정적
9,CJ,1,1,0,0,1.0,1.0,0.3,매우 긍정적


In [69]:
# 가중 감성 지수 포함 저장
weighted_results_df = pd.DataFrame(weighted_results)

# 등급 추가
weighted_results_df['등급'] = weighted_results_df['weighted_sentiment_index'].apply(get_grade)

# 저장
weighted_results_df.to_csv('../data/sentiment/weighted_sentiment_index.csv', index=False, encoding='utf-8-sig')
print("저장 완료: weighted_sentiment_index.csv")

weighted_results_df.head(10)

저장 완료: weighted_sentiment_index.csv


,stock,total,positive,negative,neutral,sentiment_index,weighted_sentiment_index,avg_weight,등급
0,SK텔레콤,1,1,0,0,1.0,1.0,0.3,매우 긍정적
1,KB금융,7,7,0,0,1.0,1.0,0.3,매우 긍정적
2,CJ ENM,1,1,0,0,1.0,1.0,0.3,매우 긍정적
3,전진건설로봇,1,1,0,0,1.0,1.0,0.3,매우 긍정적
4,NAVER,1,1,0,0,1.0,1.0,0.3,매우 긍정적
5,카카오,1,1,0,0,1.0,1.0,0.3,매우 긍정적
6,미래에셋증권,1,1,0,0,1.0,1.0,0.3,매우 긍정적
7,안랩,2,2,0,0,1.0,1.0,0.3,매우 긍정적
8,LG전자,1,1,0,0,1.0,1.0,0.3,매우 긍정적
9,CJ,1,1,0,0,1.0,1.0,0.3,매우 긍정적
